# Processing the data

Önceki bölümdeki örnekle devam edersek, PyTorch'ta bir yığın üzerinde bir dizi sınıflandırıcıyı nasıl eğiteceğimiz aşağıda açıklanmıştır:

In [1]:
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = torch.optim.AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Elbette, modeli sadece iki cümle üzerinde eğitmek çok iyi sonuçlar vermeyecektir. Daha iyi sonuçlar elde etmek için daha büyük bir veri kümesi hazırlamanız gerekecektir.

Bu bölümde örnek olarak William B. Dolan ve Chris Brockett tarafından bir makalede tanıtılan MRPC (Microsoft Research Paraphrase Corpus) veri kümesini kullanacağız. Bu veri kümesi 5.801 cümle çiftinden oluşmaktadır ve bu cümlelerin açımlama olup olmadığını (yani her iki cümlenin de aynı anlama gelip gelmediğini) gösteren bir etikete sahiptir. Bu bölüm için bu veri kümesini seçtik çünkü küçük bir veri kümesidir, bu nedenle üzerinde eğitim denemesi yapmak kolaydır.

## Loading a dataset from the Hub

Hub sadece modeller içermez; aynı zamanda birçok farklı dilde birden fazla veri kümesine sahiptir. Veri setlerine [buradan](https://huggingface.co/datasets) göz atabilirsiniz ve bu bölümden geçtikten sonra yeni bir veri seti yüklemeyi ve işlemeyi denemenizi öneririz (buradaki genel belgelere bakın). Ancak şimdilik MRPC veri setine odaklanalım! Bu, 10 farklı metin sınıflandırma görevinde makine öğrenimi modellerinin performansını ölçmek için kullanılan akademik bir ölçüt olan GLUE ölçütünü oluşturan 10 veri kümesinden biridir.

Datasets kütüphanesi, Hub'a bir veri kümesi indirmek ve önbelleğe almak için çok basit bir komut sağlar. MRPC veri kümesini şu şekilde indirebiliriz:

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

Gördüğünüz gibi, eğitim kümesi, doğrulama kümesi ve test kümesini içeren bir DatasetDict nesnesi elde ediyoruz. Bunların her biri birkaç sütun (cümle1, cümle2, etiket ve idx) ve her kümedeki öğe sayısı olan değişken sayıda satır içerir (yani, eğitim kümesinde 3.668, doğrulama kümesinde 408 ve test kümesinde 1.725 cümle çifti vardır).

Bu komut veri kümesini indirir ve varsayılan olarak ~/.cache/huggingface/datasets içinde önbelleğe alır. HF_HOME ortam değişkenini ayarlayarak önbellek klasörünüzü özelleştirebileceğinizi Bölüm 2'den hatırlayın.

raw_datasets nesnemizdeki her bir cümle çiftine bir sözlükte olduğu gibi indeksleme yaparak erişebiliriz:

In [3]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

Etiketlerin zaten tamsayı olduğunu görebiliyoruz, bu nedenle burada herhangi bir ön işlem yapmamız gerekmeyecek. Hangi tamsayının hangi etikete karşılık geldiğini bilmek için raw_train_dataset'imizin özelliklerini inceleyebiliriz. Bu bize her sütunun türünü söyleyecektir:

In [4]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

Perde arkasında, label ClassLabel türündedir ve tamsayıların label adıyla eşleştirilmesi names klasöründe saklanır. 0 not_equivalent'e karşılık gelir ve 1 equivalent'e karşılık gelir.

In [5]:
raw_train_dataset[15], raw_datasets["validation"][87]

({'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
  'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .',
  'label': 0,
  'idx': 16},
 {'sentence1': 'However , EPA officials would not confirm the 20 percent figure .',
  'sentence2': 'Only in the past few weeks have officials settled on the 20 percent figure .',
  'label': 0,
  'idx': 812})

## Preprocessing a dataset

Veri kümesini ön işleme tabi tutmak için metni modelin anlamlandırabileceği sayılara dönüştürmemiz gerekir. Önceki bölümde gördüğünüz gibi, bu bir tokenizer ile yapılır. Tokenizer bir cümle veya cümleler listesi ile besleyebiliriz, böylece her çiftin tüm ilk cümlelerini ve tüm ikinci cümlelerini aşağıdaki gibi doğrudan belirteçleyebiliriz:

In [6]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

Ancak, iki diziyi modele aktararak iki cümlenin paraphrase olup olmadığına dair bir tahmin elde edemeyiz. İki diziyi bir çift olarak ele almamız ve uygun ön işlemeyi uygulamamız gerekir. Neyse ki tokenizer da bir çift diziyi alıp BERT modelimizin beklediği şekilde hazırlayabilir:

In [7]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Bölüm 2'de input_ids ve attention_mask anahtarlarından bahsetmiştik, ancak token_type_ids hakkında konuşmayı ertelemiştik. Bu örnekte, modele girdinin hangi kısmının ilk cümle, hangisinin ikinci cümle olduğunu söyleyen şey budur.

Eğer input_ids içindeki ID'leri kelimelere geri dönüştürürsek:

In [8]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

Dolayısıyla, modelin iki cümle olduğunda girdilerin [CLS] cümle1 [SEP] cümle2 [SEP] biçiminde olmasını beklediğini görüyoruz. Bunu token_type_ids ile hizalamak bize şunu verir:

In [9]:
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[      0,      0,    0,     0,       0,          0,   0,       0,      1,    1,     1,        1,     1,   1,       1]

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

Gördüğünüz gibi, girdinin [CLS] cümle1 [SEP]'e karşılık gelen kısımlarının tümü 0 belirteç türü kimliğine sahipken, cümle2 [SEP]'e karşılık gelen diğer kısımların tümü 1 belirteç türü kimliğine sahiptir.

Farklı bir kontrol noktası seçerseniz, token_type_id'lerin tokenize edilmiş girdilerinizde olması gerekmeyeceğini unutmayın (örneğin, bir DistilBERT modeli kullanıyorsanız bunlar döndürülmez). Bunlar yalnızca model bunlarla ne yapacağını bildiğinde döndürülür, çünkü ön eğitim sırasında bunları görmüştür.

Burada BERT, belirteç türü kimlikleriyle ön eğitime tabi tutulmuştur ve Bölüm 1'de bahsettiğimiz maskeli dil modelleme hedefinin yanı sıra, sonraki cümle tahmini adı verilen ek bir hedefe sahiptir. Bu görevdeki amaç, cümle çiftleri arasındaki ilişkiyi modellemektir.

Sonraki cümle tahmini ile modele cümle çiftleri (rastgele maskelenmiş belirteçlerle) verilir ve ikinci cümlenin ilkini takip edip etmediğini tahmin etmesi istenir. Görevi önemsiz hale getirmemek için, cümleler zamanın yarısında çıkarıldıkları orijinal belgede birbirini takip eder ve diğer yarısında iki cümle iki farklı belgeden gelir.

Genel olarak, tokenize edilmiş girdilerinizde token_type_ids olup olmadığı konusunda endişelenmenize gerek yoktur: tokenizer ve model için aynı kontrol noktasını kullandığınız sürece, tokenizer modeline ne sağlayacağını bildiği için her şey yolunda gidecektir.

Artık tokenizer'ımızın bir çift cümleyle nasıl başa çıkabildiğini gördüğümüze göre, onu tüm veri setimizi tokenize etmek için kullanabiliriz: önceki bölümde olduğu gibi, tokenizer'a ilk cümlelerin listesini ve ardından ikinci cümlelerin listesini vererek cümle çiftlerinin bir listesini besleyebiliriz. Bu aynı zamanda Bölüm 2'de gördüğümüz dolgu ve kesme seçenekleriyle de uyumludur. Yani, eğitim veri kümesini önceden işlemenin bir yolu şudur:

In [10]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

Bu iyi çalışır, ancak bir sözlük döndürme dezavantajına sahiptir (anahtarlarımız, input_ids, attention_mask ve token_type_ids ve listelerin listeleri olan değerlerle). Ayrıca, yalnızca tokenizasyon sırasında tüm veri kümenizi depolamak için yeterli RAM'iniz varsa çalışacaktır (oysa Datasets kütüphanesindeki veri kümeleri diskte depolanan Apache Arrow dosyalarıdır, bu nedenle yalnızca istediğiniz örnekleri bellekte yüklü tutarsınız).

Verileri bir veri kümesi olarak tutmak için Dataset.map() yöntemini kullanacağız. Bu aynı zamanda, sadece tokenizasyondan daha fazla ön işleme ihtiyacımız olursa bize ekstra esneklik sağlar. map() metodu, veri kümesinin her bir elemanına bir fonksiyon uygulayarak çalışır, bu yüzden girdilerimizi tokenize eden bir fonksiyon tanımlayalım:

In [11]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

Bu fonksiyon bir dictionary alır (veri kümemizin öğeleri gibi) ve input_ids, attention_mask ve token_type_ids anahtarlarını içeren yeni bir sözlük döndürür. Daha önce görüldüğü gibi, tokenizer cümle çiftleri listeleri üzerinde çalıştığından, örnek sözlük birkaç örnek içeriyorsa (her anahtar bir cümle listesi olarak) da çalıştığını unutmayın. Bu, map() çağrımızda batched=True seçeneğini kullanmamıza olanak tanıyacak ve bu da tokenizasyonu büyük ölçüde hızlandıracaktır. Tokenizer,  Tokenizers kütüphanesinden Rust ile yazılmış bir tokenizer tarafından desteklenmektedir. Bu tokenizer çok hızlı olabilir, ancak yalnızca aynı anda çok sayıda girdi verirsek.

Şimdilik tokenizasyon fonksiyonumuzda padding argümanını dışarıda bıraktığımızı unutmayın. Bunun nedeni, tüm örnekleri maksimum uzunluğa kadar doldurmanın verimli olmamasıdır: bir yığın oluştururken örnekleri doldurmak daha iyidir, çünkü o zaman tüm veri kümesindeki maksimum uzunluğa değil, yalnızca o yığındaki maksimum uzunluğa kadar doldurmamız gerekir. Bu, girdiler çok değişken uzunluklara sahip olduğunda çok fazla zaman ve işlem gücü tasarrufu sağlayabilir!

Tokenlaştırma işlevini tüm veri kümelerimize aynı anda nasıl uygulayacağımız aşağıda açıklanmıştır. Map çağrımızda batched=True kullanıyoruz, böylece işlev her bir öğeye ayrı ayrı değil, veri kümemizin birden çok öğesine aynı anda uygulanıyor. Bu, daha hızlı ön işleme yapılmasını sağlar.

In [12]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

Datasets kütüphanesinin bu işlemi uygulama şekli, ön işleme fonksiyonu tarafından döndürülen sözlükteki her anahtar için bir tane olmak üzere veri kümelerine yeni alanlar eklemektir:

Hatta ön işleme fonksiyonunuzu map() ile uygularken bir num_proc argümanı ileterek çoklu işlemeyi kullanabilirsiniz. Burada bunu yapmadık çünkü Tokenizers kütüphanesi zaten örneklerimizi daha hızlı tokenize etmek için birden fazla iş parçacığı kullanıyor, ancak bu kütüphane tarafından desteklenen hızlı bir tokenizer kullanmıyorsanız, bu ön işlemenizi hızlandırabilir.

tokenize_fonksiyonumuz input_ids, attention_mask ve token_type_ids anahtarlarını içeren bir sözlük döndürür, böylece bu üç alan veri kümemizin tüm bölünmelerine eklenir. Ön işleme fonksiyonumuz map() uyguladığımız veri kümesindeki mevcut bir anahtar için yeni bir değer döndürürse mevcut alanları da değiştirebileceğimizi unutmayın.

Yapmamız gereken son şey, öğeleri bir araya getirdiğimizde tüm örnekleri en uzun öğenin uzunluğuna kadar doldurmaktır - dinamik padding olarak adlandırdığımız bir teknik.

## Dynamic padding

Örnekleri bir yığın içinde bir araya getirmekten sorumlu olan işleve collate işlevi denir. Bu, bir DataLoader oluştururken geçebileceğiniz bir argümandır, varsayılan olarak örneklerinizi PyTorch tensörlerine dönüştüren ve bunları birleştiren bir fonksiyondur (öğeleriniz listeler, tuples veya sözlükler ise özyinelemeli olarak). Elimizdeki girdilerin hepsi aynı boyutta olmayacağı için bu bizim durumumuzda mümkün olmayacaktır. Dolguyu kasıtlı olarak erteledik, böylece her bir yığında yalnızca gerektiği kadar uygulayabilir ve çok fazla dolguya sahip aşırı uzun girdilere sahip olmaktan kaçınabiliriz. Bu, eğitimi oldukça hızlandıracaktır, ancak bir TPU üzerinde eğitim yapıyorsanız bunun sorunlara neden olabileceğini unutmayın - TPU'lar, ekstra dolgu gerektirse bile sabit şekilleri tercih eder.

Bunu pratikte yapmak için, bir araya getirmek istediğimiz veri kümesinin öğelerine doğru miktarda dolgu uygulayacak bir collate işlevi tanımlamamız gerekir. Neyse ki Transformers kütüphanesi DataCollatorWithPadding aracılığıyla bize böyle bir fonksiyon sağlıyor. Bu işlev, siz onu örneklediğinizde (hangi dolgu token'ının kullanılacağını ve modelin dolgunun girdilerin solunda mı yoksa sağında mı olmasını beklediğini bilmek için) bir tokenizer alır ve ihtiyacınız olan her şeyi yapar:

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-08-07 06:23:44.934649: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 06:23:44.934787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 06:23:45.107636: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Bu yeni oyuncağı test etmek için, eğitim setimizden bir araya getirmek istediğimiz birkaç örnek alalım. Burada, idx, sentence1 ve sentence2 sütunlarını gerekmeyeceği ve dizeler içerdiği için (ve dizelerle tensör oluşturamayacağımız için) kaldırıyoruz ve yığındaki her bir girdinin uzunluklarına bakıyoruz:

In [14]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

Sürpriz değil, 32'den 67'ye kadar değişen uzunlukta örnekler alıyoruz. Dinamik dolgu, bu gruptaki örneklerin hepsinin grup içindeki maksimum uzunluk olan 67 uzunluğa kadar doldurulması gerektiği anlamına gelir. Dinamik dolgu olmadan, tüm örneklerin tüm veri kümesindeki maksimum uzunluğa veya modelin kabul edebileceği maksimum uzunluğa doldurulması gerekirdi. data_collator'ımızın yığını dinamik olarak düzgün şekilde doldurduğunu iki kez kontrol edelim:

In [15]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}